In [2]:
"""
Προσαρμόστε τη λογική απόφασης του μοντέλου της άσκησης του προηγούμενου κεφαλαίου όπου αναπτύξατε ένα IDS έτσι ώστε να αναλαμβάνει 
προληπτικές ενέργειες βάσει εντοπισμένων απειλών, προκειμένου αυτό να μετατραπεί σε IPS.
Οι ενέργειες θα μπορούσαν να περιλαμβάνουν την ενσωμάτωση κανόνων ή πολιτικών για τον καθορισμό του πότε και του τρόπου αντίδρασης 
σε εντοπισμένες εισβολές, όπως ο αποκλεισμός ύποπτης κυκλοφορίας, η απομόνωση των επηρεαζόμενων συσκευών,
η ειδοποίηση των διαχειριστών για μη αυτόματη παρέμβαση, κτλ.
"""

'\nΠροσαρμόστε τη λογική απόφασης του μοντέλου της άσκησης του προηγούμενου κεφαλαίου όπου αναπτύξατε ένα IDS έτσι ώστε να αναλαμβάνει \nπροληπτικές ενέργειες βάσει εντοπισμένων απειλών, προκειμένου αυτό να μετατραπεί σε IPS.\nΟι ενέργειες θα μπορούσαν να περιλαμβάνουν την ενσωμάτωση κανόνων ή πολιτικών για τον καθορισμό του πότε και του τρόπου αντίδρασης \nσε εντοπισμένες εισβολές, όπως ο αποκλεισμός ύποπτης κυκλοφορίας, η απομόνωση των επηρεαζόμενων συσκευών,\nη ειδοποίηση των διαχειριστών για μη αυτόματη παρέμβαση, κτλ.\n'

In [5]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
# from doqu import models_ddos
from kerastuner.tuners import RandomSearch
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Flatten
from sklearn.preprocessing import MinMaxScaler

filename='pcap_data.csv'    
trainfile = pd.read_csv(filename)
label = trainfile['target']
data = pd.DataFrame(trainfile).to_numpy()

# Standardisation
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Test data 20%
train_data, test_data, train_label, test_label = \
    train_test_split(scaled_data, label, test_size=0.20, stratify=label)
    
# Train 70%, Validation%10
train_data, val_data, train_label, val_label = \
    train_test_split(train_data, train_label,test_size=0.125, stratify=train_label)

# One-hot encode the labels
train_label = to_categorical(train_label,3)
val_label = to_categorical(val_label,3)
test_label = to_categorical(test_label,3)

# CALLBACKS 
earlyStopping = EarlyStopping(monitor='val_loss',
                              patience=30,
                              verbose=0,
                              mode='min')

modelCheckPoint = ModelCheckpoint('./savemodels/model5class.weights.{epoch:03d}-{val_accuracy:.4f}.keras',
                                  save_best_only=True,
                                  monitor='val_accuracy',
                                  mode='max')

reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1,
                                  patience=7,
                                  verbose=1,
                                  epsilon=1e-4,
                                  mode='min')


# Defining Parameters
inshape = train_data.shape[1]
lr = 1e-4
N = 64
epochs = 50
nclass = 3 # np.unique(train_label).shape[0]

# Define your neural network architecture
def create_model(lr, N, inshape):
    model = Sequential()
    model.add(Conv1D(N, kernel_size=3, activation='relu', input_shape=(inshape, 1)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nclass, activation='softmax'))  # Assuming nclass is defined elsewhere
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create the model
model = create_model(lr=lr, N=N, inshape=inshape)

# TRAIN MODEL
history = model.fit(train_data,
                    train_label,
                    shuffle=True,
                    epochs=epochs,
                    batch_size=32,  # 256,#128,#32, 64
                    # validation_data=validation_generator,
                    # validation_split=0.2,
                    validation_data=(val_data, val_label),
                    callbacks=[modelCheckPoint, earlyStopping, reduce_lr_loss])
                    # class_weight=class_weights)

# Load Best Model
str_models = os.listdir('./savemodels')
str_models = np.sort(str_models)
best_model = str_models[str_models.size-1]
print('best_model=', best_model)
model.load_weights('./savemodels/'+best_model)

# Confusion matrix 
print('TEST DATA-Confusion matrix:')  
pred = model.predict(test_data)
pred_y = pred.argmax(axis=-1)
test_label_indices = test_label.argmax(axis=-1)
cm = confusion_matrix(test_label_indices, pred_y)
print(cm)

Epoch 1/50


/var/folders/24/zglgqpr53l7_qk7br0pmq3c40000gq/T/ipykernel_36765/2798193005.py:12: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch
/Users/georgiostriantopoulos/Repos/Personal/ekpa-exercises-cybersecurity/.conda/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7992 - loss: 0.5880 - val_accuracy: 0.9980 - val_loss: 0.0221 - learning_rate: 0.0010
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9998 - loss: 0.0133 - val_accuracy: 1.0000 - val_loss: 0.0032 - learning_rate: 0.0010
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0024 - val_accuracy: 1.0000 - val_loss: 0.0012 - learning_rate: 0.0010
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 9.2571e-04 - val_accuracy: 1.0000 - val_loss: 5.8841e-04 - learning_rate: 0.0010
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 5.1979e-04 - val_accuracy: 1.0000 - val_loss: 3.5043e-04 - learning_rate: 0.0010
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 3.1033e-04 - val_accuracy: 1.0000 - val_loss: 2.2860e-04 - learning_rate: 0.0010
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - lo

In [36]:
from email.message import EmailMessage
import smtplib

def notify_admin(index, threat_type):
    msg = EmailMessage()
    msg.set_content(f"{threat_type} detected at index {index}")

    msg['Subject'] = 'Threat Detected'
    msg['From'] = 'email@example.com'
    msg['To'] = 'admin_email@example.com'

    # Uncomment the following rows and replace with valid email adreeses and credentials
    # server = smtplib.SMTP('smtp.gmail.com')
    # server.login('email@example.com', 'your_password')
    # server.send_message(msg)
    # server.quit()

def handle_threats(predictions):
    for i in range(len(predictions)):
        if (predictions[i] == 1).any():  # Anomaly
            handle_anomaly(i)
        elif (predictions[i] == 2).any():  # Attack
            handle_attack(i)

def handle_anomaly(index):
    print(f"Handling anomaly at index {index}...")
    notify_admin(index, threat_type='Anomaly')
    isolate_device(index)

def handle_attack(index):
    print(f"Handling attack at index {index}...")
    notify_admin(index, threat_type='Attack')
    isolate_device(index)
    block_traffic

def block_traffic(index):
    print('Block Traffic')
    print(f"Blocking suspicious traffic at index {index}...")

def isolate_device(index):
    print(f"Isolating affected device at index {index}...")


# after getting model's predictions
predictions = model.predict(test_data)
predictions = np.round(predictions).astype(int)
# convert the one-hot encoded values back to their original form
predictions = np.argmax(predictions, axis=1)
handle_threats(predictions)

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 515us/step
Handling attack at index 0...
Isolating affected device at index 0...
Handling attack at index 1...
Isolating affected device at index 1...
Handling anomaly at index 2...
Isolating affected device at index 2...
Handling anomaly at index 3...
Isolating affected device at index 3...
Handling attack at index 5...
Isolating affected device at index 5...
Handling attack at index 8...
Isolating affected device at index 8...
Handling anomaly at index 13...
Isolating affected device at index 13...
Handling anomaly at index 15...
Isolating affected device at index 15...
Handling attack at index 16...
Isolating affected device at index 16...
Handling anomaly at index 17...
Isolating affected device at index 17...
Handling attack at index 18...
Isolating affected device at index 18...
Handling anomaly at index 21...
Isolating affected device at index 21...
Handling anomaly at index 22...
Isolating affected device at index 22...
Handling attack at index 25.